## Summary

Calculate features using [AlphaFold](https://github.com/deepmind/alphafold).

```bash
export NOTEBOOK_PATH="$(realpath 31_run_alphafold.ipynb)"

export DATASET_NAME="cagi6-sherloc"
export DATASET_PATH="30_cagi6_sherloc/input-data-gby-protein.parquet"
export ORIGINAL_ARRAY_TASK_COUNT=4182

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=1-3500 --gres=gpu:t4:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=181-200 --gres=gpu:v100:1 ../scripts/run_notebook_gpu.sh

sbatch --export DATASET_NAME,DATASET_PATH,NOTEBOOK_PATH,ORIGINAL_ARRAY_TASK_COUNT --array=201-306 ../scripts/run_notebook_cpu.sh
```

---

## Imports

In [1]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

## Parameters

In [2]:
NOTEBOOK_DIR = Path("31_run_alphafold").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/scratch/strokach/workspace/elaspic2-cagi6/notebooks/31_run_alphafold')

In [3]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir
    
print(tempfile.gettempdir())

/tmp


In [4]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

(None, None, None, None)

In [5]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "cagi6-sherloc"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_cagi6_sherloc", "input-data-gby-protein.parquet")
    )
    TASK_ID = 1000
    TASK_COUNT = 4182
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('cagi6-sherloc',
 '/scratch/strokach/workspace/elaspic2-cagi6/notebooks/30_cagi6_sherloc/input-data-gby-protein.parquet',
 1000,
 4182)

In [6]:
device = xla_bridge.get_backend().platform

device

'gpu'

## Workspace

### Initialize model

In [7]:
AlphaFold.load_model(device=device)

### Load data

In [8]:
pfile = pq.ParquetFile(DATASET_PATH)

pfile.num_row_groups

4182

In [9]:
assert TASK_COUNT == pfile.num_row_groups

In [10]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [11]:
display(input_df.head(2))
print(len(input_df))

,protein_id,mutation_id,mutation,effect,sequence,structure,alignment
0,Q02094,[NM_000324.2:c.808G>A],[V270I],[None],MRFTFPLMAIVLEIAMIVLFGLFVEYETDQTVLEQLNITKPTDMGI...,HEADER ...,"[>101\n, MRFTFPLMAIVLEIAMIVLFGLFVEYETDQTVLEQLN..."


1


In [12]:
tup = next(input_df.itertuples(index=False))

iterable_fields = []
for field in tup._fields:
    try:
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError:
        pass

iterable_fields

['mutation_id', 'mutation', 'effect']

In [13]:
if DEBUG:
    for field in iterable_fields:
        input_df[field] = input_df[field].str[:3]

### Score mutations

In [14]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None:
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False

    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False

    return True

In [15]:
def worker(mutation, data):
    try:
        results = AlphaFold.analyze_mutation(f"A_{mutation}", data)
    except AlphaFoldAnalyzeError as error:
        print(f"{error!r}")
        return None
    else:
        results = {f"alphafold_core_{key}": value for key, value in results.items()}
        return results

In [ ]:
results = []
for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    data = AlphaFold.build(tup.sequence, ligand_sequence=None, msa=tup.alignment)

    for mutation_idx, mutation in enumerate(tqdm(tup.mutation, leave=False)):
        if not validate_mutation(mutation):
            continue

        result = worker(mutation, data)
        if result is None:
            continue

        results.append(
            {
                "protein_id": tup.protein_id,
                "mutation": mutation,
            }
            | {field: getattr(tup, field)[mutation_idx] for field in iterable_fields}
            | result
        )

results_df = pd.DataFrame(results)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpk8sh0xme.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpk8sh0xme.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpk8sh0xme.py, line 12)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpeo3c2yzo.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpeo3c2yzo.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid syntax (tmpeo3c2yzo.py, line 27)
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
display(results_df.head(2))
print(len(results_df))

### Save results

In [ ]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

In [ ]:
if not DEBUG:
    pq.write_table(pa.Table.from_pandas(results_df, preserve_index=False), output_file)